In [30]:
#here we are going to create our first agent 
#lets firstly create a tool for websearch
from langchain.tools import DuckDuckGoSearchResults 
# search_tool=DuckDuckGoSearchResults() 
# search_tool.invoke('what is machine learning')
#currently the duckduck go isn't working so are using the wikipedia retriever
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)
search_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(search_tool.invoke({"query": "What is Machine Learning"}))

Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. 
From a theoretical viewpoint, probab

In [11]:
#so it is working fine lets add the llm 
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os 

load_dotenv()
groq_api_key=os.environ['GROQ_API_KEY']
llm=ChatGroq(model='Llama3-70b-8192')

In [12]:
llm.invoke('hi')

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.074285714, 'prompt_time': 0.000284357, 'queue_time': 0.053963503, 'total_time': 0.074570071}, 'model_name': 'Llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--dfc1c4ad-686d-447b-baa7-0d5641176606-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

In [14]:
#first for creating the agent we required a prompt 
#in which we instruct llm how to work and tell about the tools and how to invoke them 
#so here we are importing from the langchain_hub
from langchain import hub 
prompt=hub.pull('hwchase17/react')

c:\Users\aashi\Envs\lang_env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
"""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}""" 

# This above is the actual prompt that we have pulled from the langchain_hub and we use it in our ReAct agent 

"input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\\n\\n{tools}\\n\\nUse the following format:\\n\\nQuestion: the input question you must answer\\nThought: you should always think about what to do\\nAction: the action to take, should be one of [{tool_names}]\\nAction Input: the input to the action\\nObservation: the result of the action\\n... (this Thought/Action/Action Input/Observation can repeat N times)\\nThought: I now know the final answer\\nFinal Answer: the final answer to the original input question\\n\\nBegin!\\n\\nQuestion: {input}\\nThought:{agent_scratchpad}'"

In [20]:
#lets create a agent 
from langchain.agents import create_react_agent ,AgentExecutor 
our_agent=create_react_agent(llm=llm,tools=[search_tool],prompt=prompt)

In [21]:
agent_execution=AgentExecutor(agent=our_agent,tools=[search_tool],verbose=True)

In [ ]:
# agent_execution.invoke('tell me which ipl team won the ipl 2025')
#above doesn't works we have to pass the userquery as the dictionary 

agent_execution.invoke({'input':'which team won the ipl 2025'})
#verbose is True this will also show the thinking of our LLM model for completing the task 



> Entering new AgentExecutor chain...
Thought: I need to find the winner of IPL 2025, but I realize that IPL 2025 has not yet occurred, so I need to find a way to answer this question differently.

Action: wikipedia
Action Input: Indian Premier LeaguePage: Indian Premier League
Summary: The Indian Premier League (IPL) is a professional Twenty20 (T20) cricket league in India, organised by the Board of Control for Cricket in India (BCCI). Founded in 2007, it features ten city-based franchise teams. The IPL is the most popular and richest cricket league, and the second-richest sports league in the world. It is held annually between March and May. It has an exclusive window in the Future Tours Programme of the International Cricket Council, resulting in fewer international tours occurring during the seasons. It is also the most viewed Indian sports event, per the Broadcast Audience Research Council.
In 2010, the IPL became the first sporting event to broadcast live on YouTube. In 2014, i

{'input': 'which team won the ipl 2025',
 'output': 'The Royal Challengers Bengaluru won the IPL 2025.'}

In [31]:
agent_execution.invoke({'input':'Tell me about the latest 2025 news'})



> Entering new AgentExecutor chain...
I need to find a reliable source of current news, but I only have access to Wikipedia, which is not ideal for getting the latest news. Wikipedia is better suited for general knowledge and historical events.

Thought: I can try searching for "2025" on Wikipedia to see if there are any notable events or articles related to the year 2025.

Action: wikipedia
Action Input: 2025Page: 2025
Summary: 2025 (MMXXV) is the current year, and is a common year starting on Wednesday of the Gregorian calendar, the 2025th year of the Common Era (CE) and Anno Domini (AD) designations, the 25th  year of the 3rd millennium and the 21st century, and the  6th   year of the 2020s decade.  
So far, the year has seen the continuation of major armed conflicts, including the Russian invasion of Ukraine, the Sudanese civil war, and the Gaza war. Internal crises in Bangladesh, Ecuador, Georgia, Germany, Haiti, Somalia, and South Korea continued into this year, with the latter

{'input': 'Tell me about the latest 2025 news',
 'output': 'According to Wikipedia, 2025 has seen the continuation of major armed conflicts, internal crises in several countries, and economic disruptions due to trade wars. There have also been significant events in the technology and aviation sectors, as well as advances in space exploration. However, please note that Wikipedia is not a reliable source for getting the latest news, and this information may not be up-to-date or comprehensive.'}

In [43]:
#lets make a simple agent in which we are including the weather api and the realtime websearch
#here we are using the webstack.com's api for gathering the weather related data of a particular place 
from langchain.tools import tool 
import requests as r 

@tool 
def Weather_tool(location: str):
    
    """This function return the Weather condition of a partcular given location"""
    api_key='bf7a779daf63abbf09092cf07e48e24b'
    url= f'https://api.weatherstack.com/current?access_key={api_key}&query={location}'
    response=r.post(url=url)
    return response.json()
Weather_tool.invoke({'location':'Alwar'}) #working fine 
    

{'request': {'type': 'City',
  'query': 'Alwar, India',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Alwar',
  'country': 'India',
  'region': 'Rajasthan',
  'lat': '27.567',
  'lon': '76.600',
  'timezone_id': 'Asia/Kolkata',
  'localtime': '2025-07-08 16:48',
  'localtime_epoch': 1751993280,
  'utc_offset': '5.50'},
 'current': {'observation_time': '11:18 AM',
  'temperature': 36,
  'weather_code': 116,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'],
  'weather_descriptions': ['Partly Cloudy '],
  'astro': {'sunrise': '05:35 AM',
   'sunset': '07:23 PM',
   'moonrise': '05:34 PM',
   'moonset': '02:54 AM',
   'moon_phase': 'Waxing Gibbous',
   'moon_illumination': 92},
  'air_quality': {'co': '288.6',
   'no2': '5.18',
   'o3': '126',
   'so2': '5.55',
   'pm2_5': '43.845',
   'pm10': '92.685',
   'us-epa-index': '3',
   'gb-defra-index': '3'},
  'wind_speed': 10,
  'wind_degree': 274,
  'wind_dir': 'W

In [44]:
agent=create_react_agent(tools=[search_tool,Weather_tool],
                         prompt=prompt,llm=llm)


In [45]:
agent_exe=AgentExecutor(agent=agent,tools=[search_tool,Weather_tool],verbose=True)


In [46]:
#lets check the working of our agent 
query='what is the captial of india and also tell me about its weather conditions' 
agent_exe.invoke({'input':query})



> Entering new AgentExecutor chain...
Thought: I need to find the capital of India, and then I need to find the current weather conditions of that city.

Action: wikipedia
Action Input: "Capital of India"Page: List of capitals of India
Summary: This is a list of locations which have served as capital cities in India. The current capital city is New Delhi, which replaced Calcutta in 1911.I know the capital of India is New Delhi.

Action: wikipedia
Action Input: "New Delhi"Page: New Delhi
Summary: New Delhi ( ; Hindi: Naī Dillī, pronounced [ˈnəiː ˈdɪlːiː]) is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court. New Delhi is a municipality within the NCT, administered by the New Delhi Municipal Council (NDMC), which covers mostly Lutyens' Delhi and a few adjacent areas. The municipal area is part of a larger administra

{'input': 'what is the captial of india and also tell me about its weather conditions',
 'output': 'The capital of India is New Delhi, and its current weather conditions are: Haze with a temperature of 35°C, humidity of 63%, wind speed of 15 km/h, and visibility of 5 km.'}

In [ ]:
#Now we can see that our agent is working fine
#THANKS FOR REACHING ME OUT IF YOU ARE READING MY CODE 
# I would like to thanks Nitish sir(founder of CampusX) for his invaluable knowledge 
